In [45]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import feature_manager as fma
import rl.env_simple_crypto_trade as env
import importlib
import rl.models as rla
import config as cf
import numpy as np
from torch.nn import Tanh
import tr_utils

In [ ]:
importlib.reload(fma)

fm = fma.FeatureManager()
fm.import_data(symbol="BTCUSDT",timeframes=["1d","1w","1mo"])

fm.build_features(
    lags = [1,1,1],
    features=cf.FOR_4H_INDICATORS,
)

In [44]:
len(fm.df)

1005

In [ ]:
importlib.reload(env)
importlib.reload(cf)

env_kwargs = cf.TRADE_ENV_PARAMETER
state_space = 10 + len(fm.cols)

full_env = env.CryptoTradingEnv(
    trade_timeframe="1d",
    df = fm.df, 
    state_space=state_space,
    indicators=fm.cols,
    **env_kwargs)

In [ ]:
len(full_env.df)

In [ ]:
importlib.reload(rla)
agent = rla.DRLTradeAgent(env=full_env)

In [ ]:
importlib.reload(rla)

# DQN_PARAMS = {
#     "learning_rate": 1e-4,
#     "buffer_size": 100_000,  
#     "learning_starts": 100_000,
#     "batch_size": 64,
#     "tau": 1.0,
#     "gamma": 0.9999,
#     "train_freq": 4,
#     "target_update_interval": 10000,
#     "exploration_fraction": 0.3,
#     "exploration_initial_eps": 1.0,
#     "exploration_final_eps": 0.05,
# }

# catalog_name = tr_utils.get_name_with_kwargs("dqn",DQN_PARAMS)

# dqn_model = agent.get_model(
#     model_name="dqn",
#     model_kwargs=DQN_PARAMS,
#     seed=100,
#     tensorboard_log=catalog_name
# )

PPO_MODEL_PARAMS = {
    "gamma": 0.99,
    "gae_lambda": 0.95,
    "n_steps": 1024,
    "ent_coef": 0.03,
    "learning_rate": 0.0002,
    "batch_size": 64
}

PPO_POLICY_PARAMS = {
    "activation_fn":Tanh,
    "net_arch": dict(pi=[128], vf=[128])
}

catalog_name = tr_utils.get_name_with_kwargs("1d_",PPO_MODEL_PARAMS)

ppo_model = agent.get_model(
    model_name="ppo",
    model_kwargs = PPO_MODEL_PARAMS,
    policy_kwargs= PPO_POLICY_PARAMS,
    tensorboard_log=catalog_name,
    seed = 100
)


In [ ]:
catalog_name

In [ ]:
ppo_model.policy_kwargs

In [43]:
selected_model = ppo_model

selected_model = agent.train_model(
    model = selected_model,
    total_timesteps = 1_000_000,
    checkpoint = True,
    catalog_name = catalog_name,
    save_frequency = 20_000,
    progress_bar = True
)

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1e+03         |
|    ep_rew_mean          | 0.489         |
| time/                   |               |
|    fps                  | 164           |
|    iterations           | 89            |
|    time_elapsed         | 553           |
|    total_timesteps      | 91136         |
| train/                  |               |
|    approx_kl            | 0.00027597573 |
|    clip_fraction        | 0.00889       |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.232        |
|    explained_variance   | 0.00154       |
|    learning_rate        | 0.0002        |
|    loss                 | -0.000962     |
|    n_updates            | 880           |
|    policy_gradient_loss | -0.000453     |
|    reward               | 0.0           |
|    value_loss           | 0.0184        |
-------------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1e+03        |
|    ep_rew_mean          | 0.49         |
| time/                   |              |
|    fps                  | 164          |
|    iterations           | 91           |
|    time_elapsed         | 566          |
|    total_timesteps      | 93184        |
| train/                  |              |
|    approx_kl            | 0.0007260723 |
|    clip_fraction        | 0.0112       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.19        |
|    explained_variance   | 0.000504     |
|    learning_rate        | 0.0002       |
|    loss                 | 0.0141       |
|    n_updates            | 900          |
|    policy_gradient_loss | -0.00178     |
|    reward               | 0.0040146452 |
|    value_loss           | 0.0324       |
------------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1e+03        |
|    ep_rew_mean          | 0.493        |
| time/                   |              |
|    fps                  | 164          |
|    iterations           | 92           |
|    time_elapsed         | 571          |
|    total_timesteps      | 94208        |
| train/                  |              |
|    approx_kl            | 0.0010669219 |
|    clip_fraction        | 0.0181       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.197       |
|    explained_variance   | 0.000348     |
|    learning_rate        | 0.0002       |
|    loss                 | 0.0224       |
|    n_updates            | 910          |
|    policy_gradient_loss | -0.00196     |
|    reward               | 0.0          |
|    value_loss           | 0.0508       |
------------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1e+03        |
|    ep_rew_mean          | 0.497        |
| time/                   |              |
|    fps                  | 164          |
|    iterations           | 93           |
|    time_elapsed         | 578          |
|    total_timesteps      | 95232        |
| train/                  |              |
|    approx_kl            | 0.0012172242 |
|    clip_fraction        | 0.0145       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.204       |
|    explained_variance   | 0.000125     |
|    learning_rate        | 0.0002       |
|    loss                 | 0.00991      |
|    n_updates            | 920          |
|    policy_gradient_loss | -0.000922    |
|    reward               | 0.0          |
|    value_loss           | 0.0248       |
------------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1e+03        |
|    ep_rew_mean          | 0.503        |
| time/                   |              |
|    fps                  | 164          |
|    iterations           | 94           |
|    time_elapsed         | 584          |
|    total_timesteps      | 96256        |
| train/                  |              |
|    approx_kl            | 0.0011271774 |
|    clip_fraction        | 0.0154       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.206       |
|    explained_variance   | 0.00057      |
|    learning_rate        | 0.0002       |
|    loss                 | -0.0012      |
|    n_updates            | 930          |
|    policy_gradient_loss | -0.00196     |
|    reward               | 0.025579385  |
|    value_loss           | 0.023        |
------------------------------------------


...

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1e+03         |
|    ep_rew_mean          | 0.814         |
| time/                   |               |
|    fps                  | 164           |
|    iterations           | 974           |
|    time_elapsed         | 6062          |
|    total_timesteps      | 997376        |
| train/                  |               |
|    approx_kl            | 0.00055274327 |
|    clip_fraction        | 0.0156        |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.265        |
|    explained_variance   | 0.000729      |
|    learning_rate        | 0.0002        |
|    loss                 | 0.0066        |
|    n_updates            | 9730          |
|    policy_gradient_loss | -0.000672     |
|    reward               | 0.0047642374  |
|    value_loss           | 0.0152        |
-------------------------------------------


-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1e+03         |
|    ep_rew_mean          | 0.81          |
| time/                   |               |
|    fps                  | 164           |
|    iterations           | 975           |
|    time_elapsed         | 6069          |
|    total_timesteps      | 998400        |
| train/                  |               |
|    approx_kl            | 0.00074176764 |
|    clip_fraction        | 0.0129        |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.233        |
|    explained_variance   | 0.000407      |
|    learning_rate        | 0.0002        |
|    loss                 | 0.00289       |
|    n_updates            | 9740          |
|    policy_gradient_loss | -0.00194      |
|    reward               | 0.0           |
|    value_loss           | 0.0291        |
-------------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1e+03        |
|    ep_rew_mean          | 0.806        |
| time/                   |              |
|    fps                  | 164          |
|    iterations           | 976          |
|    time_elapsed         | 6076         |
|    total_timesteps      | 999424       |
| train/                  |              |
|    approx_kl            | 0.0009774028 |
|    clip_fraction        | 0.0107       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.271       |
|    explained_variance   | 0.000226     |
|    learning_rate        | 0.0002       |
|    loss                 | 0.00426      |
|    n_updates            | 9750         |
|    policy_gradient_loss | 5.24e-05     |
|    reward               | 0.036227506  |
|    value_loss           | 0.0193       |
------------------------------------------


-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1e+03         |
|    ep_rew_mean          | 0.806         |
| time/                   |               |
|    fps                  | 164           |
|    iterations           | 977           |
|    time_elapsed         | 6083          |
|    total_timesteps      | 1000448       |
| train/                  |               |
|    approx_kl            | 0.00052788906 |
|    clip_fraction        | 0.0124        |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.266        |
|    explained_variance   | 0.000662      |
|    learning_rate        | 0.0002        |
|    loss                 | 0.00102       |
|    n_updates            | 9760          |
|    policy_gradient_loss | -0.000216     |
|    reward               | 0.0073506264  |
|    value_loss           | 0.0224        |
-------------------------------------------


Total trained timestep: 1000448


In [ ]:
agent.predict(selected_model,full_env,render=True)